In [2]:
import pandas as pd
import numpy as np

In [3]:
policia = {'estado': np.random.randint(1,3, size=30),
           'id_caso': range(0,30,1),
           'fecha': np.random.randint(1,12, size=30),
           'categoria': np.random.choice(list(['robos', 'hurto', 'muerte', 'incendio']), 30)}
df = pd.DataFrame(policia)

In [4]:
df['trim'] = df['fecha']%4
# df1 = df.loc[df['estado'] == 2].loc[df['trim'].isin([1,2])]
df1 = df.groupby(['categoria', 'trim']).agg({'id_caso': 'count'}).reset_index()
df1

,categoria,trim,id_caso
0,hurto,0,2
1,hurto,1,1
2,hurto,2,4
3,hurto,3,2
4,incendio,1,3
5,incendio,2,1
6,incendio,3,1
7,muerte,0,4
8,muerte,1,2
9,muerte,2,1


In [5]:
df3 = df1.pivot_table(values='id_caso', columns='trim', index='categoria')
df3.loc[df3[1] < df3[2]][[1,2]].index.values.tolist()
# df3

['hurto']

In [1]:
import pyspark
try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
type(sc)

pyspark.context.SparkContext

In [63]:
policia = [(20190123,20,3,'hurto'), (20190126,21,1,'hurto'), (20190221,22,2,'hurto'),
           (20190401,28,2,'hurto'), (20190702,31,3,'hurto'), (20190313,30,2,'hurto'),
           (20190610,27,2,'hurto'),
           (20190526,24,2,'robo'), (20190412,25,2,'robo'), (20190723,26,2,'robo'), 
           (20190301,30,2,'robo'), (201900523,29,1,'robo'),
            ]

batman = [(20, 1), (21, 1), (22, 1), (28, 0), (31, 0), (30, 1), (27, 1),
            (24, 1), (25, 0), (26, 1), (30, 0), (29, 1)]
batman = sc.parallelize(batman);
data = sc.parallelize(policia);

In [17]:
def trimestre(fecha):
    return 1 if fecha>20190401 else 0

In [39]:
resuelto =2
d1 = data.filter(lambda x: x[0] >= 20190101 and x[0] < 20190701)\
            .map(lambda x: ((trimestre(x[0]), x[3]), x[2]))\
            .groupByKey()\
            .mapValues(list)
d1.collect()

[((0, 'hurto'), [3, 1, 2, 2, 2]),
 ((1, 'robo'), [2, 2]),
 ((1, 'hurto'), [2]),
 ((0, 'robo'), [2])]

In [51]:
d2 = d1.map(lambda x: (x[0][1], (x[0][0], sum(1 for i in x[1] if i == 2) / len(x[1]))))
d2.collect()

[('hurto', (0, 0.6)),
 ('robo', (1, 1.0)),
 ('hurto', (1, 1.0)),
 ('robo', (0, 1.0))]

In [60]:
def filter_(x,y):
    if (x[0]>y[0] and x[1]>= (1.1*y[1])):
        return (x)
    if (y[0]>x[0] and y[1]>= (1.1*x[1])):
        return (y)
        
# esto es horrible

In [62]:
d2.reduceByKey(filter_).filter(lambda x: x[1] != None).map(lambda x: x[0]).collect()

['hurto']

In [101]:
policia = data.map(lambda x: (x[1], x[3]))\
                .join(batman)\
                .map(lambda x: (x[1]))\
                .groupByKey()\
                .mapValues(list)\
                .map(lambda x: (x[0], sum(1 for i in x[1] if i == 1) / len(x[1])))

In [100]:
policia.collect()

[('robo', 0.6666666666666666), ('hurto', 0.625)]